In [2]:
# what I want
# I want a model that 
# translates all the things I ask into another language

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [32]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [33]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for i in range(0, len(history), 2):
        human = history[i]
        assistant = history[i+1] if i+1 < len(history) else None

        if human['role'] == 'user':
            messages.append({"role": "user", "content": human['content']})
        if assistant and assistant['role'] == 'assistant':
            messages.append({"role": "assistant", "content": assistant['content']})
    messages.append({"role" : "user" , "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

In [97]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7891

To create a public link, set `share=True` in `launch()`.


In [34]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination):
    city = destination.lower()
    return ticket_prices.get(city, "Unknown")

In [35]:
get_ticket_price("LondoN")

'$799'

In [36]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination. Call this whenever you need to know the ticket price, for example when you're asked 'how much is a ticket to x?'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "the city that customer wants to travel to"
            },
        },
        "required": ["destination"],
        "additionalProperties": False
    }
}

In [37]:
tools = [{"type": "function", "function": price_function}]

In [38]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for i in range(0, len(history), 2):
        human = history[i]
        assistant = history[i+1] if i+1 < len(history) else None

        if human['role'] == 'user':
            messages.append({"role": "user", "content": human['content']})
        if assistant and assistant['role'] == 'assistant':
            messages.append({"role": "assistant", "content": assistant['content']})
    messages.append({"role" : "user" , "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(mode=MODEL, messages=messages)
    return response.choices[0].message.content

In [39]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    args = json.loads(tool_call.function.arguments)
    city = args.get('destination')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination" : city, "price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [106]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_nqrofDvzpdSprefoh7nCv6JQ', function=Function(arguments='{"destination":"Tokyo"}', name='get_ticket_price'), type='function')])
ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hfnZU7s00POmZ3C0vWLmo1KQ', function=Function(arguments='{"destination":"America"}', name='get_ticket_price'), type='function')])


In [40]:
from PIL import Image, ImageDraw, ImageFont

In [41]:
def create_ticket(destination: str, passenger: str, price: str, filename: str = "week2Ticket.png"):
    width, height = 600, 300
    background_color = (255, 255, 255)
    text_color = (0, 0, 0)
    border_color = (0, 0, 0)

    ticket = Image.new('RGB', (width, height), background_color)
    draw = ImageDraw.Draw(ticket)
    border_thickness = 10
    draw.rectangle([(0, 0), (width, height)], outline=border_color, width=border_thickness)

    try:
        font = ImageFont.truetype("arial.ttf", 40)
    except IOError:
        font = ImageFont.load_default()

    destination_text = f"Destination: {destination}"
    destination_bbox = draw.textbbox((0, 0), destination_text, font=font)
    destination_size = (destination_bbox[2] - destination_bbox[0], destination_bbox[3] - destination_bbox[1])
    draw.text(((width - destination_size[0]) / 2, 70), destination_text, font=font, fill=text_color)

    
    passenger_text = f"Passenger: {passenger}"
    passenger_bbox = draw.textbbox((0, 0), passenger_text, font=font)
    passenger_size = (passenger_bbox[2] - passenger_bbox[0], passenger_bbox[3] - passenger_bbox[1])
    draw.text(((width - passenger_size[0]) / 2, 140), passenger_text, font=font, fill=text_color)

    
    price_text = f"Price: ${price}"
    price_bbox = draw.textbbox((0, 0), price_text, font=font)
    price_size = (price_bbox[2] - price_bbox[0], price_bbox[3] - price_bbox[1])
    draw.text(((width - price_size[0]) / 2, 210), price_text, font=font, fill=text_color)

    file_path = f"{filename}"
    ticket.save(file_path)
    return file_path

In [42]:
ticket_path = create_ticket("America", "Amir", "1999.99")

In [43]:
book_function = {
    "name": "create_ticket",
    "description": "Give the png image for ticket for each passenger with destination that you have the price for. Call this whenever you need to book the ticket, but if you don't know the passenger name, ask for it. for example when a customer says 'Book a ticket for a specific destination', you may ask about his / her name and then create the ticket",
    "parameters": {
        "type": "object",
        "properties": {
            "destination": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "passenger_name": {
                "type": "string",
                "description": "The customer name that wants to travel",
            },
        },
        "required": ["destination","passenger_name"],
        "additionalProperties": False
    }
}

In [44]:
tools = [{"type": "function", "function": price_function},{"type": "function", "function": book_function}]

In [45]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    destination = arguments.get('destination')
    if (tool_call.function.name == 'create_ticket'):
        passenger_name = arguments.get('passenger_name')
        price = get_ticket_price(destination)
        file_path = create_ticket(destination,passenger_name, price)
        response = {
            "role": "tool",
            "content": json.dumps({"file_path": file_path}),
            "tool_call_id": message.tool_calls[0].id
        }
    elif (tool_call.function.name == 'get_ticket_price'):
        price = get_ticket_price(destination)
        response = {
            "role": "tool",
            "content": json.dumps({"destination": destination, "price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
    return response, destination

In [46]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for i in range(0, len(history), 2):
        human = history[i]
        assistant = history[i+1] if i+1 < len(history) else None

        if human['role'] == 'user':
            messages.append({"role": "user", "content": human['content']})
        if assistant and assistant['role'] == 'assistant':
            messages.append({"role": "assistant", "content": assistant['content']})
    messages.append({"role" : "user" , "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [121]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7897

To create a public link, set `share=True` in `launch()`.


In [47]:
def translate_ticket(ticket_data,lang="it"):
    ticket_text = "\n".join([f"{key}: {value}" for key, value in ticket_data.items()])
    
    prompt = (
        f"Translate the following ticket details into {lang}. "
        f"Ensure that all names, dates, and venue information are properly translated:\n\n{ticket_text}"
    )

    try:
        response = openai.chat.completions.create(model=MODEL,messages=[{"role": "user", "content": prompt}])
        translated_text = response.choices[0].message.content
    
        translated_data = {}
        for line in translated_text.split("\n"):
            if ": " in line:
                key, value = line.split(": ", 1)
                translated_data[key.strip()] = value.strip()
        return translated_data

    except Exception as e:
        print("Error during translation:", e)
        return None

In [48]:
ticket_data = {
    "event_name": "Concert",
    "date": "2024-12-05",
    "venue": "Teatro Regio",
    "seat": "A10"
}

translated_ticket = translate_ticket(ticket_data, "fr")
print(translated_ticket)

{"nom_de_l'événement": 'Concert', 'date': '2024-12-05', 'lieu': 'Teatro Regio', 'place': 'A10'}


In [114]:
translate_function = {
    "name": "translate_ticket",
    "description": "Translate the given English ticket data to the requested language. When the customer requests a translated ticket, return a ticket in the specified language using this function.",
    "parameters": {
        "type": "object",
        "properties": {
            "ticket_data": {
                "type": "object",
                "description": "The complete ticket data in English, to be translated.",
            },
            "language": {
                "type": "string",
                "description": "The language to translate the ticket to, e.g., 'French' for a French translation.",
            },
        },
        "required": ["ticket_data", "language"],
        "additionalProperties": False
    }
}

In [115]:
tools = [{"type": "function", "function": price_function},{"type": "function", "function": book_function}, {"type": "function", "function": translate_function}]

In [116]:
from PIL import Image, ImageDraw, ImageFont

def create_custom_png(data, output_path="Week2Ticket2.png"):
    width, height = 600, 300
    background_color = (245, 245, 245)
    text_color = (0, 0, 0)
    line_color = (200, 200, 200)

    ticket_image = Image.new("RGB", (width, height), background_color)
    draw = ImageDraw.Draw(ticket_image)

    try:
        font = ImageFont.truetype("arial.ttf", 18)
        header_font = ImageFont.truetype("arial.ttf", 24)
    except IOError:
        font = ImageFont.load_default()
        header_font = font

    header_text = "Event Ticket"
    header_bbox = draw.textbbox((0, 0), header_text, font=header_font)
    header_w = header_bbox[2] - header_bbox[0]
    draw.text(((width - header_w) // 2, 20), header_text, fill=text_color, font=header_font)

    draw.line([(50, 60), (width - 50, 60)], fill=line_color, width=2)

    y_position = 80
    padding = 10

    for key, value in ticket_data.items():
        key_text = f"{key}:"
        value_text = str(value)

        key_bbox = draw.textbbox((0, 0), key_text, font=font)
        key_w = key_bbox[2] - key_bbox[0]
        draw.text((50, y_position), key_text, fill=text_color, font=font)
        draw.text((50 + key_w + padding, y_position), value_text, fill=text_color, font=font)

        y_position += key_bbox[3] - key_bbox[1] + padding

    border_color = (150, 150, 150)
    for i in range(0, width, 10):
        draw.line([(i, 0), (i + 5, 0)], fill=border_color, width=2)
        draw.line([(i, height - 1), (i + 5, height - 1)], fill=border_color, width=2)
    for i in range(0, height, 10):
        draw.line([(0, i), (0, i + 5)], fill=border_color, width=2)
        draw.line([(width - 1, i), (width - 1, i + 5)], fill=border_color, width=2)

    ticket_image.save(output_path)

In [117]:
create_custom_png(translated_ticket)

In [118]:
def handle_tool_call(message):
    global stored_destination
    global stored_passenger_name
    
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    destination = arguments.get('destination')
    
    if (tool_call.function.name == 'create_ticket'):
        passenger_name = arguments.get('passenger_name')
        stored_passenger_name = passenger_name
        stored_destination = destination
        price = get_ticket_price(destination)
        file_path = create_ticket(destination,passenger_name, price)
        response = {
            "role": "tool",
            "content": json.dumps({"file_path": file_path}),
            "tool_call_id": message.tool_calls[0].id
        }
        
    elif (tool_call.function.name == 'get_ticket_price'):
        price = get_ticket_price(destination)
        stored_destination = destination
        response = {
            "role": "tool",
            "content": json.dumps({"destination": destination, "price": price}),
            "tool_call_id": message.tool_calls[0].id
        }
        
    elif (tool_call.function.name == 'translate_ticket'):
        data = {
            "destination": stored_destination,
            "passenger": stored_passenger_name,
            "price": get_ticket_price(stored_destination)
        }
        
        file_path = translate_ticket(data)
        response = {
            "role": "tool",
            "content": json.dumps({"file_path": file_path}),
            "tool_call_id": message.tool_calls[0].id
        }
    return response, destination

In [119]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for i in range(0, len(history), 2):
        human = history[i]
        assistant = history[i+1] if i+1 < len(history) else None

        if human['role'] == 'user':
            messages.append({"role": "user", "content": human['content']})
        if assistant and assistant['role'] == 'assistant':
            messages.append({"role": "assistant", "content": assistant['content']})
    messages.append({"role" : "user" , "content" : message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [120]:
gr.ChatInterface(fn=chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
